<a href="https://colab.research.google.com/github/csabi0312/crypto-svm/blob/pandas_ta/crypto16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pandas_ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 KB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218923 sha256=1b20d216b4b29f0f57c0be966d651d610564a42ba85d0461d6749758c01be516
  Stored in directory: /root/.cache/pip/wheels/54/4a/75/06b8e63fce6f6d2c1baae5c208edb18eca128407b0c96e1153
Successfully built pandas_ta


In [4]:
!pip install oandapyV20

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.6/51.6 KB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for oandapyV20: filename=oandapyV20-0.7.2-py3-none-any.whl size=70035 sha256=12aad2968dc1abd6e384b71489aa70aa11a23122fa17ec60746ddab4773964b8
  Stored in directory: /root/.cache/pip/wheels/d2/b6/8e/6fe7d26410a99ddb24ac4358691c5feabf4320d20d8053ca25
Successfully built oandapyV20


In [6]:
!pip install pandas_ta --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [57]:
import pandas as pd
import pandas_ta as ta
import oandapyV20
import oandapyV20.endpoints.instruments as instruments
import lightgbm as lgb
from sklearn.metrics import classification_report, confusion_matrix
# Set Oanda API credentials
accountID = "your_account_ID"
access_token = '9d53d7108a5cf9dd067d645999b09fb2-add25b05c95da90348b85621df9c2ca5'
client = oandapyV20.API(access_token=access_token, headers={"Accept-Datetime-Format":"RFC3339"})

# Set instrument and time frame
instrument = "BTC_USD"
granularity = "D"

# Get historical data from Oanda
params = {
    "count": 5000,
    "granularity": granularity
}
r = instruments.InstrumentsCandles(instrument=instrument, params=params)
data = client.request(r)

# Convert data to pandas DataFrame
df = pd.DataFrame([candle['mid'] for candle in data['candles']])
df['volume']=[candle['volume'] for candle in data['candles']]
df['time'] = [candle['time'] for candle in data['candles']]
df['time'] = pd.to_datetime(df['time'])
df = df.set_index('time')  # set 'time' column as index
df = df.rename(columns={'c': 'close', 'o': 'open', 'l': 'low','h':"high"})
df['close'] = pd.to_numeric(df['close'])  # convert 'close' column to numerical type
df['low'] = pd.to_numeric(df['low'])
df['high'] = pd.to_numeric(df['high'])
df['open'] = pd.to_numeric(df['open'])
df['volume'] = pd.to_numeric(df['volume'])

# Generate technical analysis features with pandas_ta
df.ta.rsi(length=14)
df.ta.roc(length=12)
df.ta.cci(length=20)
df.ta.pgo()
df.ta.macd(fast=12, slow=26, signal=9)
df.ta.kama(length=10, pow1=2, pow2=30)
df.ta.sma(length=10)
df.ta.sma(length=20)
df.ta.sma(length=50)
df.ta.sma(length=100)
df.ta.sma(length=200)
df.ta.ema(length=10)
df.ta.ema(length=20)
df.ta.ema(length=50)
df.ta.ema(length=100)
df.ta.ema(length=200)
df.ta.dema(length=10)
df.ta.dema(length=20)
df.ta.dema(length=50)
df.ta.dema(length=100)
df.ta.dema(length=200)
df.ta.tema(length=10)
df.ta.tema(length=20)
df.ta.tema(length=50)
df.ta.tema(length=100)
df.ta.tema(length=200)
df.ta.psar()
df.ta.stoch(length=14, smooth=3)
df.ta.stochrsi(length=14, smoothK=3, smoothD=3)
df.ta.bbands(length=20, std=2)
df.ta.kc(length=20, scalar=2)
df.ta.psar()
df.ta.supertrend(length=10, multiplier=3)
df.ta.adx(length=14)
df.ta.adx(length=14)
df.ta.ao()
df.ta.aroon(length=14)
df.ta.atr(length=14)
df.ta.mfi(length=14)
df.ta.vortex(length=14)
df.ta.ui(length=14)
df.ta.ichimoku()

df['daily_return'] = (df['close'].pct_change()>=0).astype(int)

# Drop NaN values
df.dropna(inplace=True)

# Split data into training and testing sets
split_ratio = 0.8
split_index = int(split_ratio * len(df))
X_train, y_train = df.iloc[:split_index, :-1], df.iloc[:split_index, -1]
X_test, y_test = df.iloc[split_index:, :-1], df.iloc[split_index:, -1]

# Train LightGBM model
model = lgb.LGBMRegressor(n_estimators=150)
model.fit(X_train, y_train)

# Make predictions on test set
y_pred = (model.predict(X_test)>=0.5).astype(int)

# Evaluate model performance
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[139  80]
 [ 83 122]]
              precision    recall  f1-score   support

           0       0.63      0.63      0.63       219
           1       0.60      0.60      0.60       205

    accuracy                           0.62       424
   macro avg       0.62      0.61      0.61       424
weighted avg       0.62      0.62      0.62       424



In [53]:
df

,open,high,low,close,volume,daily_return
time,,,,,,
2016-01-02 22:00:00+00:00,437.5,437.8,426.5,427.0,18334,-0.024000
2016-01-03 22:00:00+00:00,423.3,436.5,423.3,435.5,18443,0.019906
2016-01-04 22:00:00+00:00,435.5,436.5,427.8,433.5,36292,-0.004592
2016-01-05 22:00:00+00:00,433.5,433.7,427.1,431.2,28355,-0.005306
2016-01-06 22:00:00+00:00,431.2,457.0,428.7,456.4,39521,0.058442
...,...,...,...,...,...,...
2023-02-12 22:00:00+00:00,21707.0,21901.5,21360.6,21630.8,242809,0.002624
2023-02-13 22:00:00+00:00,21630.2,22333.6,21544.5,22255.8,241249,0.028894
2023-02-14 22:00:00+00:00,22254.2,24362.2,22051.5,24180.5,213735,0.086481
